# Data Export

In [2]:
import pymysql
import pandas
import re
import tqdm
import sqlalchemy
import PIL.Image
import os
import matplotlib.pyplot as plt 
import boto3
import multiprocessing
import io
import datetime
# from io import BytesIO
# from multiprocessing import Process, Manager
# import psycopg2
# import requests

In [3]:
class counter:

    def __init__(self, user, password, server, database):

        self.user = user
        self.password = password
        self.server = server
        self.database = database
        return

    def connect(self):

        information = [self.user, self.password, self.server, self.database]
        url = 'mysql+pymysql://{}:{}@{}/{}'.format(*information)
        self.connection = sqlalchemy.create_engine(url).connect()
        return

    def disconnect(self):

        self.connection.close()
        return

    def read(self, query):

        execution = self.connection.execute(query)
        table = pandas.DataFrame(execution.fetchall())
        return(table)

    pass

In [4]:
checkpoint = "temporary storage"
os.makedirs(checkpoint, exist_ok=True)

In [5]:
user = 'admin'
password = 'dermai_root_1234'
server = "dermai.clhzpxphx83l.ap-northeast-1.rds.amazonaws.com"
database = 'moleUp'

In [6]:
engine = counter(user, password, server, database)
engine.connect()

In [7]:
date   = "'2022-10-01 00:00:00'"
query  = "SELECT * FROM `results` where create_time < {};".format(date)
label  = engine.read(query)
print(label.shape)
label.head()

(10780, 12)


,rowid,dataset,image,user_id,image_name,score,prediction,dr,dr_yoe,result,times,create_time
0,1,MoleMe,226,Uc6ab328b0d792092b57f7b285028ead2,Uc6ab328b0d792092b57f7b285028ead2-202106071105...,0.9882624745368958,Higher,greg,,skip,2021-06-07 03:39:34,2021-06-07 03:39:34
1,2,MoleMe,229,Uc6ab328b0d792092b57f7b285028ead2,Uc6ab328b0d792092b57f7b285028ead2-202106071217...,0.984902560710907,Higher,greg,,skip,2021-06-07 05:07:58,2021-06-07 05:07:58
2,3,MoleMe,232,U4e7c8abd9fb0ea631296a31d2772f2da,U4e7c8abd9fb0ea631296a31d2772f2da-202106071627...,0.5069690346717834,Lower,greg,1,skip,2021-06-08 03:10:19,2021-06-08 03:10:19
3,4,MoleMe,231,U4e7c8abd9fb0ea631296a31d2772f2da,U4e7c8abd9fb0ea631296a31d2772f2da-202106071625...,0.39824753999710083,Lower,greg,1,skip,2021-06-08 03:10:21,2021-06-08 03:10:21
4,5,MoleMe_OA,48,U199c77fc82b06db1a0e682abf9ee2507,U199c77fc82b06db1a0e682abf9ee2507-xck3a0b6-201...,0.6862580180168152,Lower,jack,1,higher,2021-06-11 00:57:57,2021-06-11 00:57:57


In [8]:
node = str(datetime.datetime.now()).split(' ')[-1].replace(":", "-").replace('.', '-')
link = os.path.join(checkpoint, 'label_{}.csv'.format(node))
label.to_csv(link, index=False)

In [9]:
query  = "SELECT * FROM `doedoe`;"
question  = engine.read(query)
column = [
    'user_id', 'timestamp', 'image', 
    'partition', "howlong", "squeeze", 
    "cream", "medicine", "menstruation", 
    "age", "sex", "prediction", 
    "probability", "threshold", "source"
]
question = question[column]
print(question.shape)
question.head()

(2788, 15)


,user_id,timestamp,image,partition,howlong,squeeze,cream,medicine,menstruation,age,sex,prediction,probability,threshold,source
0,Ud6f9b1aa136ec2ef9a9e14642a390cf2,2021-02-09 04:49:51.470906,Ud6f9b1aa136ec2ef9a9e14642a390cf2-202102091026...,0,0,0,0,0,1,0,1,1,0.6356186866760254,0.5,PEIEN
1,Ud6f9b1aa136ec2ef9a9e14642a390cf2,2021-02-09 04:51:06.436264,Ud6f9b1aa136ec2ef9a9e14642a390cf2-202102091250...,2,1,1,3,1,1,0,1,0,0.29209890961647034,0.5,PEIEN
2,Ud6f9b1aa136ec2ef9a9e14642a390cf2,2021-02-09 05:05:19.373971,Ud6f9b1aa136ec2ef9a9e14642a390cf2-202102091251...,2,1,1,1,1,None,2,0,0,0.15067601203918457,0.5,PEIEN
3,Ud6f9b1aa136ec2ef9a9e14642a390cf2,2021-02-24 04:02:05.003698,Ud6f9b1aa136ec2ef9a9e14642a390cf2-202102241201...,2,1,1,1,1,2,2,1,0,0.4583189785480499,0.5,PEIEN
4,U37b8aadb85d302f6f68bd918fa18f397,2021-02-24 14:47:42.696500,U37b8aadb85d302f6f68bd918fa18f397-202102242247...,1,0,0,1,1,None,2,0,0,0.4811391532421112,0.5,ASKiN


In [10]:
question['partition'] = question['partition'].replace({"0":"額頭區域", "1":"鼻子區域", "2":"臉頰區域", "3":"嘴巴、下巴區域", "4":"我不確定"})
question['howlong'] = question['howlong'].replace({"0":"三個月以下", "1":"三個月以上"})
question['squeeze'] = question['squeeze'].replace({"0":"有", "1":'沒有'})
question['cream'] = question['cream'].replace({"0":"有，醫師開立的外用藥", "1":"有，藥局購買的外用藥", "2":"有，非藥用抗痘產品", "3":"沒有"})
question['medicine'] = question['medicine'].replace({"0":"是", "1":"否"})
question['age'] = question['age'].replace({"0":"18歲以下", "1":"19-25歲", "2":"26-40歲", "3":"41-50歲", "4":"51歲以上"})
question['sex'] = question['sex'].replace({"0":"男性", "1":"女性", "2":"不想回答"})
question['menstruation'] = question['menstruation'].replace({"0":"是", "1":"否", "2":"我不確定", None:"性別男性跳過"})
question.head(3)

,user_id,timestamp,image,partition,howlong,squeeze,cream,medicine,menstruation,age,sex,prediction,probability,threshold,source
0,Ud6f9b1aa136ec2ef9a9e14642a390cf2,2021-02-09 04:49:51.470906,Ud6f9b1aa136ec2ef9a9e14642a390cf2-202102091026...,額頭區域,三個月以下,有,有，醫師開立的外用藥,是,否,18歲以下,女性,1,0.6356186866760254,0.5,PEIEN
1,Ud6f9b1aa136ec2ef9a9e14642a390cf2,2021-02-09 04:51:06.436264,Ud6f9b1aa136ec2ef9a9e14642a390cf2-202102091250...,臉頰區域,三個月以上,沒有,沒有,否,否,18歲以下,女性,0,0.29209890961647034,0.5,PEIEN
2,Ud6f9b1aa136ec2ef9a9e14642a390cf2,2021-02-09 05:05:19.373971,Ud6f9b1aa136ec2ef9a9e14642a390cf2-202102091251...,臉頰區域,三個月以上,沒有,有，藥局購買的外用藥,否,性別男性跳過,26-40歲,男性,0,0.15067601203918457,0.5,PEIEN


In [11]:
node = str(datetime.datetime.now()).split(' ')[-1].replace(":", "-").replace('.', '-')
link = os.path.join(checkpoint, 'question_{}.csv'.format(node))
question.to_csv(link, index=False)

---